# For now, I will try to plot synthetic data

In [1]:
# %conda install -c conda-forge plotly -y

In [2]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go 
import plotly.io as pio
import rpy2.robjects as robjects
from plotly.subplots import make_subplots
# from rpy2.robjects import numpy2ri
# from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage


In [3]:
%matplotlib inline

## Reference cells

In [4]:
# # Generate nicely looking random 3D-field
# np.random.seed(0)
# l = 30
# X, Y, Z = np.mgrid[:l, :l, :l]
# vol = np.zeros((l, l, l))
# pts = (l * np.random.rand(3, 15)).astype(np.int)
# vol[tuple(indices for indices in pts)] = 1
# from scipy import ndimage
# vol = ndimage.gaussian_filter(vol, 4)
# vol /= vol.max()

# fig = go.Figure(data=go.Volume(
#     x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
#     value=vol.flatten(),
#     isomin=0.2,
#     isomax=0.7,
#     opacity=0.1,
#     surface_count=25,
#     ))
# fig.update_layout(scene_xaxis_showticklabels=False,
#                   scene_yaxis_showticklabels=False,
#                   scene_zaxis_showticklabels=False)
# fig.show()

In [5]:
# # Create a grid of x, y values
# x = np.linspace(-5, 5, 100)
# y = np.linspace(-5, 5, 100)
# x, y = np.meshgrid(x, y)

# # Define a function to generate z values
# z = np.sin(np.sqrt(x ** 2 + y ** 2))

# surface = go.Surface(x=x, y=y, z=z, colorscale= 'Cividis')

# layout = go.Layout(
#     title='3D Surface Plot',
#     scene=dict(
#         xaxis_title='X Axis',
#         yaxis_title='Y Axis',
#         zaxis_title='Z Axis'
#     )
# )

# fig = go.Figure(data=[surface], layout=layout)
# fig.show()

Data import

In [6]:
# #importing saved inouts and predictions from R

# pathToFile = '../R/results/3D-model/predictedVals.csv'

# #import data to df from path in csv format
# df = pd.read_csv(pathToFile)

# df.head()

# Importing the models. 

In [7]:
%ls ../R

Boehmite_LM.r             bin/                      proto_model.rds
Data transformation code/ data/                     results/
Imogolite_LM.r            doc/                      short_imo.rds
Interaction_3d.R          guinierFits.R             si_model.RDS
Phase_map - draft.R       lm_grid.rds               silica_LM.R
Phase_map.R               long_imo.rds              src/
Phase_map_current.R       long_stdrd_imo.rds        tibble_allPhases.rds
Rad_LM.R                  phaseMap.rds
bh_model.RDS              proto_LM.R


In [8]:
# Load the R model from the .rds file
path_imo = "../R/long_imo.rds"
path_bh = "../R/bh_model.RDS"
path_pImo = "../R/proto_model.rds"
path_si = "../R/si_model.RDS"
# robjects.r["load"](path_imo)

# Load the R object from the environment
model_imo = robjects.r['readRDS'](path_imo)
model_bh = robjects.r['readRDS'](path_bh)
model_pImo = robjects.r['readRDS'](path_pImo)
model_si = robjects.r['readRDS'](path_si)

# Access the R model functions and attributes
print(model_imo)
print(model_bh)
print(model_pImo)
print(model_si)


Call:
lm(formula = rt_imo ~ I(log(concentration)) + baseRatio + baseRate + 
    cationRatio + baseRatio:baseRate + cationRatio:baseRate + 
    I(baseRatio^2), data = imogolite)

Coefficients:
          (Intercept)  I(log(concentration))              baseRatio  
             -14.9912                -2.0985                 6.4219  
             baseRate            cationRatio         I(baseRatio^2)  
               0.8895                 1.5741                -1.7174  
   baseRatio:baseRate   baseRate:cationRatio  
              -0.2350                -0.3140  



Call:
lm(formula = I(boehmite^(1/3)) ~ (I(log(concentration)) * cationRatio), 
    data = bh)

Coefficients:
                      (Intercept)              I(log(concentration))  
                          -8.0698                            -1.9890  
                      cationRatio  I(log(concentration)):cationRatio  
                           6.0691                             0.9793  



Call:
lm(formula = p.Imo ~ (baseRa

The code below specifies the range of conditions I'm working with

In [9]:
# Create 1D arrays for input conditions
concentration = np.linspace(0.005, 0.2, 10)
baseRatio = np.linspace(0.5, 3, 10)
cationRatio = np.linspace(1, 2, 10)

# Create a meshgrid from the 1D arrays
conc_mesh, base_mesh,cat_mesh  = np.meshgrid(concentration, baseRatio, cationRatio)

#fixed baseRate
rate_mesh = np.ones(conc_mesh.shape) * 2


# # Convert meshgrid arrays to R format
# r_conc_mesh = pandas2ri.py2ri(conc_mesh)
# r_base_mesh = pandas2ri.py2ri(base_mesh)
# r_cat_mesh = pandas2ri.py2ri(cat_mesh)




### The code below is used to provide predictions based on the model for **imogolite**

In [10]:

# Coefficients from the linear regression model
coefficients_imo = {
    'Intercept': -14.9912,
    'I(log(concentration))': -2.0985,
    'baseRatio': 6.4219,
    'baseRate': 0.8895,
    'cationRatio': 1.5741,
    'I(baseRatio^2)': -1.7174,
    'baseRatio:baseRate': -0.2350,
    'baseRate:cationRatio': -0.3140
}

# Calculate predicted values using the coefficients and input data
predictions_imo = (
    coefficients_imo['Intercept'] +
    coefficients_imo['I(log(concentration))'] * np.log(conc_mesh) +
    coefficients_imo['baseRatio'] * base_mesh +
    coefficients_imo['baseRate'] * rate_mesh +
    coefficients_imo['cationRatio'] * cat_mesh +
    coefficients_imo['I(baseRatio^2)'] * (base_mesh ** 2) +
    coefficients_imo['baseRatio:baseRate'] * (base_mesh * rate_mesh) +
    coefficients_imo['baseRate:cationRatio'] * (rate_mesh * cat_mesh)
)

# accounting for transformation and limiting the range of the predictions
predictions_imo = predictions_imo ** 3
predictions_imo = np.clip(predictions_imo, 0, 100)




### The code below is used to provide predictions based on the model for **pseudo-boehmite**

In [11]:

# Given coefficients from the linear regression model
coefficients_bh = {
    'Intercept': -8.0698,
    'I(log(concentration))': -1.9890,
    'cationRatio': 6.0691,
    'I(log(concentration)):cationRatio': 0.9793
}


# Calculate predicted values using the coefficients and input data
predictions_bh = (
    coefficients_bh['Intercept'] +
    coefficients_bh['I(log(concentration))'] * np.log(conc_mesh) +
    coefficients_bh['cationRatio'] * cat_mesh +
    coefficients_bh['I(log(concentration)):cationRatio'] * (np.log(conc_mesh) * cat_mesh)
)

# accounting for transformation and limiting the range of the predictions
predictions_bh = predictions_bh ** 3
predictions_bh = np.clip(predictions_bh, 0, 100)




### The code below is used to provide predictions based on the model for **proto-imogolite**

In [12]:

# Coefficients from the linear regression model
coefficients_pImo = {
    'Intercept': -14.9912,
    'I(log(concentration))': -2.0985,
    'baseRatio': 6.4219,
    'baseRate': 0.8895,
    'cationRatio': 1.5741,
    'I(baseRatio^2)': -1.7174,
    'baseRatio:baseRate': -0.2350,
    'baseRate:cationRatio': -0.3140
}

# Calculate predicted values using the coefficients and input data
predictions_pImo = (
    coefficients_imo['Intercept'] +
    coefficients_imo['I(log(concentration))'] * np.log(conc_mesh) +
    coefficients_imo['baseRatio'] * base_mesh +
    coefficients_imo['baseRate'] * rate_mesh +
    coefficients_imo['cationRatio'] * cat_mesh +
    coefficients_imo['I(baseRatio^2)'] * (base_mesh ** 2) +
    coefficients_imo['baseRatio:baseRate'] * (base_mesh * rate_mesh) +
    coefficients_imo['baseRate:cationRatio'] * (rate_mesh * cat_mesh)
)

# Given coefficients from the linear regression model
coefficients_pImo = {
    'Intercept': -2.511,
    'baseRate': -4.397,
    'cationRatio': 28.938,
    'baseRatio': 14.201,
    'I(log(concentration))': -45.952,
    'I((log(concentration))^2)': -7.814,
    'cationRatio:baseRatio': -22.811,
    'baseRate:baseRatio': 2.543
}

# Calculate predicted values using the coefficients and input data
predictions_pImo = (
    coefficients_pImo['Intercept'] +
    coefficients_pImo['baseRate'] * rate_mesh +
    coefficients_pImo['cationRatio'] * cat_mesh +
    coefficients_pImo['baseRatio'] * base_mesh +
    coefficients_pImo['I(log(concentration))'] * np.log(conc_mesh) +
    coefficients_pImo['I((log(concentration))^2)'] * (np.log(conc_mesh) ** 2) +
    coefficients_pImo['cationRatio:baseRatio'] * (cat_mesh * base_mesh) +
    coefficients_pImo['baseRate:baseRatio'] * (rate_mesh * base_mesh)
)


# accounting for transformation and limiting the range of the predictions
predictions_pImo = np.clip(predictions_pImo, 0, 100)




### The code below is used to provide predictions based on the model for **amorphous silica**

In [13]:

# Given coefficients from the linear regression model
coefficients_si = {
    'Intercept': 8.4147,
    'I(log(concentration))': 1.1296,
    'baseRatio': -4.2802,
    'cationRatio': -0.3560,
    'I(baseRatio^2)': 0.6753,
    'I(log(concentration)):baseRatio': -0.4080
}

# Calculate predicted values using the coefficients and input data
predictions_si = (
    coefficients_si['Intercept'] +
    coefficients_si['I(log(concentration))'] * np.log(conc_mesh) +
    coefficients_si['baseRatio'] * base_mesh +
    coefficients_si['cationRatio'] * cat_mesh +
    coefficients_si['I(baseRatio^2)'] * (base_mesh ** 2) +
    coefficients_si['I(log(concentration)):baseRatio'] * (np.log(conc_mesh) * base_mesh)
)


# accounting for transformation and limiting the range of the predictions
predictions_si = predictions_si ** 3
predictions_si = np.clip(predictions_si, 0, 100)




Finally, the 3D plot is displayed here

In [14]:
# predictions = np.sum([predictions_imo.flatten(), predictions_pImo.flatten()], axis=0)

# Create a 3D surface plot using Plotly
# plotting the imogolite model first
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}]],
           vertical_spacing=0.1,
           horizontal_spacing=0.1)

fig.add_trace(go.Volume(z=cat_mesh.flatten(),
                                 x=conc_mesh.flatten(), 
                                 y=base_mesh.flatten(), 
                                 value=predictions_imo.flatten(),
                                # value=predictions.flatten(),
                                 isomin=50,
                                 isomax=100,
                                 opacity=0.5,
                                 surface_count=10),
                                 row=1, col=1)

fig.add_trace(go.Volume(z=cat_mesh.flatten(),
                                 x=conc_mesh.flatten(), 
                                 y=base_mesh.flatten(), 
                                 value=predictions_bh.flatten(),
                                # value=predictions.flatten(),
                                 isomin=50,
                                 isomax=100,
                                 opacity=0.5,
                                 surface_count=10), 
                                 row=1, col=2)
fig.add_trace(go.Volume(z=cat_mesh.flatten(),
                                 x=conc_mesh.flatten(), 
                                 y=base_mesh.flatten(), 
                                 value=predictions_pImo.flatten(),
                                # value=predictions.flatten(),
                                 isomin=50,
                                 isomax=100,
                                 opacity=0.5,
                                 surface_count=10),
                                 row=2, col=1)

fig.add_trace(go.Volume(z=cat_mesh.flatten(),
                                 x=conc_mesh.flatten(), 
                                 y=base_mesh.flatten(), 
                                 value=predictions_si.flatten(),
                                # value=predictions.flatten(),
                                 isomin=50,
                                 isomax=100,
                                 opacity=0.5,
                                 surface_count=10),
                                 row=2, col=2)

fig.update_scenes(
    xaxis_title='',
    yaxis_title='',
    zaxis_title='',
    xaxis_type='log',
    yaxis_type='linear',
    zaxis_type='linear',
    xaxis_showticklabels=False,
    yaxis_showticklabels=False,
    zaxis_showticklabels=False,
)
# #axis labels
# fig.update_layout(scene=dict(zaxis=dict(title='Cation Ratio', type='linear')))
# fig.update_layout(scene=dict(xaxis=dict(title='Concentration', type='log')))
# fig.update_layout(scene=dict(yaxis=dict(title='Hydrolysis ratio', type='linear')))


# Show the plot
fig.show()

In [15]:
#this saves the final image
# image_path = 'volumeSubplot.png'
# pio.write_image(fig, image_path, format='png', width=600, height=600, scale=5)


### And this is what the final image looks like:
<img src='./volumeSubplot.png'>